# GSM-EPD-LIH

In [ ]:
%matplotlib widget
import matplotlib.pylab as plt
import sympy as sp
import numpy as np
from IPython.display import display, Math, Markdown
from bmcs_matmod.api import GSM1D_EPD
gsm = GSM1D_EPD()
display(Markdown(gsm.latex_potentials()))

## Monotonic strain-driven loading

In [ ]:
mparams = dict(E = 20, K = 100, S = 30, c = 1, r = 1, eps_0 = 0, f_c = 15, eta_vp=40)
n_t = 1510
#n_t = 11
n_I = 1
eps_max = 1.3
final_time = 3
t_t = np.linspace(0, final_time, n_t)
eps_n_t = np.linspace(0, eps_max, n_t)
response = gsm.get_response(eps_n_t, t_t, **mparams)
t_t_ec, eps_ta_ec, sig_ta_ec, Eps_t_ec, Sig_t_ec, iter_t_ec, lam_t_ec, (d_t_t_ec, d_eps_ta_ec) = response
eps_t_ec = eps_ta_ec[:, 0]
sig_t_ec = sig_ta_ec[:, 0, 0]
# use the stress history to reproduce the response using stress control
response = gsm.get_G_response(sig_t_ec, t_t, **mparams)
t_t_sc, eps_ta_sc, sig_ta_sc, Eps_t_sc, Sig_t_sc, iter_t_sc, lam_t_sc, (d_t_t_sc, d_eps_ta_sc) = response
iter_t = iter_t_sc[:, 0]
eps_t_sc = eps_ta_sc[:, 0, 0]
sig_t_sc = sig_ta_sc[:, 0]
eps_t_ec.shape, sig_t_ec.shape, eps_t_sc.shape, sig_t_sc.shape

In [ ]:
eps_p_t_ec, omega_t_ec, z_t = Eps_t_ec[:, 0, :].T
sig_p_t_ec, Y_t_ec, Z_t_ec = Sig_t_ec[:, 0, :].T
eps_p_t_sc, omega_t_sc, z_t_sc = Eps_t_sc[:, 0, :].T
sig_p_t_sc, Y_t_sc, Z_t_sc = Sig_t_sc[:, 0, :].T

In [ ]:
fig, (ax_sig, ax_eps) = plt.subplots(1, 2, figsize=(12, 4))

ax_sig.plot(eps_t_ec, sig_t_ec, color='cadetblue', label=r'$\varepsilon$')
ax_sig.plot(eps_t_ec-eps_p_t_ec, sig_t_ec, ls='dashed', color='cadetblue', label=r'$\varepsilon-\varepsilon^\mathrm{ve}$')

ax_sig.plot(eps_t_sc, sig_t_sc, color='red', label=r'$\varepsilon$')
ax_sig.plot(eps_t_sc-eps_p_t_sc, sig_t_sc, ls='dashed', color='red', label=r'$\varepsilon-\varepsilon^\mathrm{ve}$')

ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend()

ax_eps.plot(t_t_ec, eps_t_ec, label=r'$\varepsilon$', color='cadetblue', lw=1)
ax_eps.plot(t_t_ec, eps_p_t_ec, label=r'$\varepsilon_\mathrm{vp}$', color='cadetblue', ls='dashed', lw=1)

ax_eps.plot(t_t_sc, eps_t_sc, label=r'$\varepsilon$', color='red', lw=1)
ax_eps.plot(t_t_sc, eps_p_t_sc, label=r'$\varepsilon_\mathrm{vp}$', color='red', ls='dashed', lw=1)
ax_eps.legend()
ax_eps.set_ylabel(r'$\varepsilon$', color='cadetblue')

plt.tight_layout()
plt.show()

## Cyclic stress-driven loading

In [ ]:
t_, sig_max_, t_1_, t_2_, t_3_ = sp.symbols('t sig_max t_1 t_2 t_3', real=True, positive=True)
eps_t_ = sp.Piecewise(
    (sig_max_ / t_1_ * t_, t_ < t_1_),
    (sig_max_, t_ < t_2_),
    (sig_max_ - sig_max_/(t_3_-t_2_)*(t_ - t_2_), t_ < t_3_),
    (0, True)
)
get_eps_t = sp.lambdify((t_, sig_max_, t_1_, t_2_, t_3_), eps_t_, 'numpy')
sig_max_val = np.max(sig_t_ec)
n_t_cyclic = 4 * n_t
final_time_cyclic = 4 * final_time 
t_t = np.linspace(0, final_time_cyclic, n_t_cyclic)
sig_n_t = get_eps_t(t_t, sig_max_val, 0.25*final_time_cyclic, 0.5*final_time_cyclic, 0.75*final_time_cyclic)

# Extend the loading history with 10 alternating sections
for i in range(1, 2):
    t_t = np.hstack((t_t, t_t[-1] + t_t[1:]))
    sig_n_t = np.hstack([sig_n_t, (-1)**i * sig_n_t[1:]])

for i in range(1, 2):
    t_t = np.hstack((t_t, t_t[-1] + t_t[1:]))
    sig_n_t = np.hstack([sig_n_t, sig_n_t[1:]])

fig, ax = plt.subplots(1, 1)
ax.plot(t_t, sig_n_t)

In [ ]:
response = gsm.get_G_response(sig_n_t, t_t, **mparams)
t_t, eps_ta, sig_ta, Eps_t, Sig_t, iter_t, lam_t, (d_t_t, d_eps_ta) = response
iter_t = iter_t[:, 0]
eps_t_sc_cyclic = eps_ta[:, 0, 0]
sig_t_sc_cyclic = sig_ta[:, 0]
eps_p_t_sc_cyclic, omega_t_sc_cyclic, z_t_sc_cyclic = Eps_t[:, 0, :].T
sig_p_t_sc_cyclic, Y_t_sc_cyclic, Z_t_sc_cyclic = Sig_t[:, 0, :].T
eps_t_sc_cyclic.shape, sig_t_sc_cyclic.shape

In [ ]:
fig, ((ax_sig, ax_eps), (ax_lam, ax_z)) = plt.subplots(2, 2, figsize=(12, 8))

# First row: Stress-strain and damage plots
ax_iter = ax_sig.twinx()
ax_omega = ax_eps.twinx()
ax_sig_t = ax_lam.twinx()

# fig, (ax_sig, ax_eps) = plt.subplots(1, 2, figsize=(12, 4))

# Plot stress-strain and iterations-strain curves
# ax_iter = ax_sig.twinx()

ax_sig.plot(eps_t_sc_cyclic, sig_t_sc_cyclic, color='cadetblue', label=r'$\varepsilon$')
# ax_sig.plot(eps_t_sc_cyclic-eps_p_t_sc_cyclic, sig_t_sc_cyclic, ls='dashed', color='cadetblue', label=r'$\varepsilon-\varepsilon^\mathrm{ve}$')
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')

ax_sig.plot(eps_t_sc, sig_t_sc, color='orange', label=r'$\varepsilon$')
# ax_sig.plot(eps_t_sc-eps_p_t_sc, sig_t_sc, ls='dashed', color='orange', label=r'$\varepsilon-\varepsilon^\mathrm{ve}$')
ax_sig.legend()

ax_iter.plot(eps_t_sc_cyclic, iter_t, ls='dashed', color='gray', lw=0.5)
ax_iter.set_ylabel(r'$k$', color='gray')

ax_eps.plot(t_t, eps_t_sc_cyclic, label=r'$\varepsilon$', color='cadetblue', lw=1)
ax_eps.plot(t_t, eps_p_t_sc_cyclic, label=r'$\varepsilon_\mathrm{vp}$', color='cadetblue', ls='dashed', lw=1)
ax_eps.plot(t_t_sc, eps_t_sc, label=r'$\varepsilon$', color='orange', lw=1)
ax_eps.plot(t_t_sc, eps_p_t_sc, label=r'$\varepsilon_\mathrm{vp}$', color='orange', ls='dashed', lw=1)
ax_omega.plot(t_t, omega_t_sc_cyclic, label=r'$\omega$', color='red')
ax_omega.set_ylabel(r'$\omega$', color='red')
ax_omega.set_xlabel(r'$t$')
ax_omega.legend()

ax_eps.plot(t_t, eps_t_sc_cyclic-eps_p_t_sc_cyclic, label=r'$\varepsilon-\varepsilon_\mathrm{vp}$', color='magenta', ls='dashed', lw=1)

ax_lam.plot(t_t, lam_t[:, 0, 0], label=r'$\lambda$', color='green')
ax_sig_t.plot(t_t, sig_n_t, label=r'$\sigma$', color='orange', ls='dashed', lw=1)

ax_eps.legend()
ax_eps.set_ylabel(r'$\varepsilon$', color='cadetblue')

plt.tight_layout()
plt.show()

## Cyclic strain-driven loading

In [ ]:
t_, sig_max_, t_1_, t_2_, t_3_ = sp.symbols('t sig_max t_1 t_2 t_3', real=True, positive=True)
eps_t_ = sp.Piecewise(
    (sig_max_ / t_1_ * t_, t_ < t_1_),
    (sig_max_, t_ < t_2_),
    (sig_max_ - sig_max_/(t_3_-t_2_)*(t_ - t_2_), t_ < t_3_),
    (0, True)
)
get_eps_t = sp.lambdify((t_, sig_max_, t_1_, t_2_, t_3_), eps_t_, 'numpy')
eps_max = 2
n_t_ec_cyclic = 4 * n_t
f_time = 4 * final_time
t_t = np.linspace(0, f_time, n_t_ec_cyclic)
eps_n_t = get_eps_t(t_t, eps_max, 0.25*f_time, 0.5*f_time, 0.75*f_time)
fig, ax = plt.subplots(1,1)
ax.plot(t_t, eps_n_t)

In [ ]:
mparams = dict(E = 20, K = 5, S = 10, c = 1, r = 1, eps_0 = 0, f_c = 15, eta_vp=1)
response = gsm.get_response(eps_n_t, t_t, **mparams)
t_t, eps_ta, sig_ta, Eps_t, Sig_t, iter_t, lam_t, (d_t_t, d_eps_ta) = response
# Plot viscoplastic strain and damage along time
eps_t = eps_ta[:, 0]
sig_t = sig_ta[:, 0]
eps_p_t, omega_t, z_t = Eps_t[:, 0, :].T
sig_p_t, Y_t, Z_t = Sig_t[:, 0, :].T

In [ ]:
fig, ((ax_sig, ax_eps), (ax_lam, ax_z)) = plt.subplots(2, 2, figsize=(12, 8))

# First row: Stress-strain and damage plots
ax_iter = ax_sig.twinx()
ax_omega = ax_eps.twinx()

ax_sig.plot(eps_t, sig_t, color='cadetblue', label=r'$\varepsilon$')
# ax_sig.plot(eps_t-eps_p_t, Sig_t[:,:,0], ls='dashed', color='cadetblue', label=r'$\varepsilon-\varepsilon^\mathrm{ve}$')
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend()

ax_iter.plot(t_t, iter_t, ls='dashed', color='gray', lw=0.5)
ax_iter.set_ylabel(r'$k$', color='gray')

ax_eps.plot(t_t, eps_t, label=r'$\varepsilon$', color='cadetblue', lw=1)
ax_eps.plot(t_t, eps_p_t, label=r'$\varepsilon_\mathrm{vp}$', color='cadetblue', ls='dashed', lw=1)
ax_eps.legend()
ax_eps.set_ylabel(r'$\varepsilon$', color='cadetblue')
ax_omega.plot(t_t, omega_t, label=r'$\omega$', color='red')
ax_omega.set_ylabel(r'$\omega$', color='red')
ax_omega.set_xlabel(r'$t$')
ax_omega.legend()

# Second row: Lagrange multiplier and plastic strain plots
ax_lam_eps_t = ax_lam.twinx()
ax_lam.plot(t_t, lam_t[:, 0, 0], label=r'$\lambda$', color='purple')
ax_lam.set_xlabel(r'$t$')
ax_lam.set_ylabel(r'$\lambda$', color='purple')
ax_lam_eps_t.plot(t_t, eps_t, label=r'$\varepsilon$', color='green', ls='dashed')
ax_lam_eps_t.set_ylabel(r'$\varepsilon$', color='green')

ax_z_eps_p = ax_z.twinx()
ax_z.plot(t_t, Z_t, label=r'$z$', color='orange')
ax_z.set_xlabel(r'$t$')
ax_z.set_ylabel(r'$Z$', color='orange')
ax_z_eps_p.plot(t_t, eps_p_t, label=r'$\varepsilon_\mathrm{p}$', color='blue', ls='dashed')
ax_z_eps_p.set_ylabel(r'$\varepsilon_\mathrm{p}$', color='blue')

plt.tight_layout()
plt.show()